## Captain Learning Pytorch
Learning: https://pytorch.org/

In [15]:
import torch
print(torch.__version__)

2.5.1.post108


In [16]:
# pytorch tensors

tensor1 = torch.tensor([1, 2, 3])             # From a list
tensor2 = torch.zeros(3, 4)                   # 3x4 matrix of zeros
tensor3 = torch.rand(2, 3)                    # Random numbers
tensor4 = torch.eye(3)                        # Identity matrix
# key properties
print(tensor1.shape)      # Shape
print(tensor2.dtype)      # Data type
print(tensor3.device)     # Device (CPU/GPU)



torch.Size([3])
torch.float32
cpu


In [17]:
# Tensor Operations and manipulation
a = torch.tensor([2, 4, 6])
b = torch.tensor([1, 3, 5])

add = a + b               # Addition
sub = a - b               # Subtraction
mul = a * b               # Element-wise multiplication
div = a / b               # Division
print(add)
print(sub)
print(mul)
print(div)

# reshape
tensor = torch.arange(12)       # [0, 1, 2, ..., 11]
reshaped = tensor.view(3, 4)    # Reshape to 3x4 matrix
flattened = reshaped.flatten()  # Flatten back to 1D

#moving data between gpu and cpu
# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move tensor to GPU
tensor = torch.tensor([1, 2, 3]).to(device)


tensor([ 3,  7, 11])
tensor([1, 1, 1])
tensor([ 2, 12, 30])
tensor([2.0000, 1.3333, 1.2000])


In [18]:
# Autograd
x = torch.tensor(2.0, requires_grad=True)  # Track gradient
print(x)
y = x ** 3                                 # y = x³
y.backward()                               # Backpropagation

print(x.grad)                              # dy/dx = 3x² = 12


tensor(2., requires_grad=True)
tensor(12.)


In [19]:
# building a neural network
import torch.nn as nn
import torch.nn.functional as F

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(4, 8)    # Input: 4 features → 8 neurons
        self.fc2 = nn.Linear(8, 3)    # Hidden layer → 3 output classes

    def forward(self, x):
        x = F.relu(self.fc1(x))       # ReLU activation
        x = self.fc2(x)               # Output layer
        return x

model = NeuralNetwork()


In [20]:
# training neural network:
import torch.optim as optim

# Data (dummy)
X = torch.rand(10, 4)         # 10 samples, 4 features
y = torch.randint(0, 3, (10,)) # 10 labels for 3 classes

# Loss function & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
for epoch in range(100):
    outputs = model(X)                  # Forward pass
    loss = criterion(outputs, y)       # Compute loss

    optimizer.zero_grad()              # Clear gradients
    loss.backward()                    # Backpropagation
    optimizer.step()                   # Update weights

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')


Epoch 0, Loss: 1.1840566396713257
Epoch 10, Loss: 1.1555649042129517
Epoch 20, Loss: 1.1322376728057861
Epoch 30, Loss: 1.1121034622192383
Epoch 40, Loss: 1.0954350233078003
Epoch 50, Loss: 1.0806498527526855
Epoch 60, Loss: 1.0678431987762451
Epoch 70, Loss: 1.056889295578003
Epoch 80, Loss: 1.0474046468734741
Epoch 90, Loss: 1.040174126625061


In [ ]:
# optimizer
#optimizer = optim.Adam(model.parameters(), lr=0.001)
#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)


In [22]:
# loss functions
#criterion = nn.CrossEntropyLoss()  # For multi-class classification
#print(criterion(model(X), y).item())

1.0339409112930298


In [23]:
# using a dataset
from torch.utils.data import DataLoader, TensorDataset

# Dataset
dataset = TensorDataset(X, y)
loader = DataLoader(dataset, batch_size=4, shuffle=True)

# Training with DataLoader
for batch in loader:
    inputs, labels = batch
    outputs = model(inputs)


In [24]:
# Model Evaluations and Metrics
correct = 0
total = 0

with torch.no_grad():  # No gradient calculation during evaluation
    for inputs, labels in loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Accuracy: {accuracy * 100:.2f}%')


Accuracy: 40.00%


In [25]:
# confusion matrix and F1 Score
from sklearn.metrics import confusion_matrix, f1_score

# Convert tensors to NumPy
y_true = labels.numpy()
y_pred = predicted.numpy()

# Metrics
print(confusion_matrix(y_true, y_pred))
print(f'F1 Score: {f1_score(y_true, y_pred, average="macro")}')


[[0 0 1]
 [0 0 0]
 [0 1 0]]
F1 Score: 0.0


In [ ]:
# Transfer learning and pretrained models
from torchvision import models

# Load a pretrained ResNet
model = models.resnet18(pretrained=True)

# Freeze layers (optional)
for param in model.parameters():
    param.requires_grad = False

# Modify the final layer for new classification task
model.fc = nn.Linear(512, 2)  # Adjusting output for 2 classes


In [ ]:
# Gpu acceleration with Cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Move model and data to GPU
model.to(device)
X, y = X.to(device), y.to(device)


In [26]:
torch.save(model.state_dict(), 'model.pth')
model.load_state_dict(torch.load('model.pth'))
model.eval()  # Set to evaluation mode


/var/folders/b_/bx4v7gvs0pd2mj6yzb1x41m40000gn/T/ipykernel_18960/1320286838.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model.pth')

NeuralNetwork(
  (fc1): Linear(in_features=4, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=3, bias=True)
)